### Webscrape two tables from website and convert to pandas dataframe
#### one table has the location data, the other has the PFAS sample data

In [1]:
import requests
import os
import json
from requests import get
import pickle
import pandas as pd
import numpy as np
import re

os.chdir(r'/Users/swilson/Library/CloudStorage/OneDrive-Personal/python/python notebooks/GitHub/Webscraping')
cwd = os.getcwd()
print("Current working directory is:", cwd)

Current working directory is: /Users/swilson/Library/CloudStorage/OneDrive-Personal/python/python notebooks/GitHub/Webscraping


In [2]:
#making request using request package and viewing json
#pulling in the metadata which as latitude and longitude for each military base, which is needed for future mapping
meta_url = 'https://providers.acq.osd.mil/pfasapi/odata/installation?'
metadata = requests.get(meta_url)
metadata = metadata.json()

In [3]:
#converting from json to pandas dataframe
metadata_df = pd.DataFrame.from_dict(metadata['value'])

#there is at least one duplicate, Letterkenney, so removing duplicates
metadata_df = metadata_df.sort_values('InstallationName', ascending=True).drop_duplicates('InstallationName')

metadata_df

#optional storing metadata for later use
#%store metadata_df

,Id,DodComponentId,DodComponent,State,InstallationName,InstallationType,Latitude,Longitude
0,1,2100,Army,Virgin Islands,AAOF Blair Hangar,National Guard,17.725062,-64.782945
1,2,2100,Army,Alabama,AASF #1 R W Shepherd Hope Hull,National Guard,32.287216,-86.394073
2,3,2100,Army,Alabama,AASF #2 Birmingham,National Guard,33.572333,-86.750542
3,4,2100,Army,Tennessee,AASF #2 TN,National Guard,35.821234,-83.994949
4,5,2100,Army,Washington,AASF #2 WA,National Guard,47.618166,-117.641352
...,...,...,...,...,...,...,...,...
696,698,5700,Air Force,Michigan,Wurtsmith AFB,BRAC,44.452430,-83.380475
699,701,1700,Navy,Arizona,YUMA AZ MCAS,Active,32.645334,-114.602265
697,699,5700,Air Force,West Virginia,Yeager (McLaughlin),National Guard,38.372966,-81.591703
698,700,5700,Air Force,Ohio,Youngstown Air Reserve Station,Active,41.257562,-80.667245


In [4]:
#pull in raw sample data
sample_url = 'https://providers.acq.osd.mil/pfasapi/odata/installation?$select=Id,DodComponent,DodComponentId,InstallationName,InstallationType,State&expand=FinalData'
sample_data = requests.get(sample_url)
sample_data_json = sample_data.json()
sample_data_df = pd.DataFrame.from_records(sample_data_json['value'])
sample_data_df

,Id,DodComponentId,DodComponent,State,InstallationName,InstallationType,FinalData,FinalData@odata.nextLink
0,1,2100,Army,Virgin Islands,AAOF Blair Hangar,National Guard,[],NaN
1,2,2100,Army,Alabama,AASF #1 R W Shepherd Hope Hull,National Guard,[],NaN
2,3,2100,Army,Alabama,AASF #2 Birmingham,National Guard,[],NaN
3,4,2100,Army,Tennessee,AASF #2 TN,National Guard,[],NaN
4,5,2100,Army,Washington,AASF #2 WA,National Guard,[],NaN
...,...,...,...,...,...,...,...,...
702,704,2100,Army,Indiana,Jefferson Proving Ground,Active,[],NaN
703,705,2100,Army,Texas,ASF Conroe,Active,[],NaN
704,706,1700,Navy,Alaska,WALES AK,Active,[],NaN
705,707,1700,Navy,Washington,EVERETT WA NAVSTA,Active,[],NaN


In [5]:
#accessing sublist called "Final Data"
sublist_final = sample_data_df.loc[:,['FinalData']]#.head()
sublist_final

,FinalData
0,[]
1,[]
2,[]
3,[]
4,[]
...,...
702,[]
703,[]
704,[]
705,[]


In [6]:
for i in sublist_final.columns:
    mod_final=sublist_final.explode(i)
mod_final

,FinalData
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
702,NaN
703,NaN
704,NaN
705,NaN


In [7]:
#convert final list to pandas dataframe
flat_dct=pd.json_normalize(json.loads(mod_final.to_json(orient="records")))

#pd.set_option('display.max_rows', 5000)
flat_dct.head()

,FinalData,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.LabReportId,FinalData.LaboratorySampleId,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.CasNumber,FinalData.AnalyteAbbrev,FinalData.FinalResult,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.FinalResultDate,FinalData.LkupInstallationId,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#drop rows that are empty and FinalData column which is empty
final_data_nona = flat_dct[flat_dct['FinalData.InstallationName'].notna()]
final_data_nona = final_data_nona.drop('FinalData', axis=1)
final_data_nona.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.LabReportId,FinalData.LaboratorySampleId,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.CasNumber,FinalData.AnalyteAbbrev,FinalData.FinalResult,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.FinalResultDate,FinalData.LkupInstallationId,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
23,123460.0,Air Force,Air Force Plant 44,Active,FA99631,FA99631-1,2022-10-11T00:00:00-04:00,QSM_B15,757124-72-4,4:2FTS,,40,ng/L,UJ,1900-01-01T00:00:00-05:00,24.0,Yes,pre-treatment
24,123461.0,Air Force,Air Force Plant 44,Active,FA99631,FA99631-1,2022-10-11T00:00:00-04:00,QSM_B15,27619-97-2,6:2FTS,,8,ng/L,U,1900-01-01T00:00:00-05:00,24.0,Yes,pre-treatment
25,123462.0,Air Force,Air Force Plant 44,Active,FA99631,FA99631-1,2022-10-11T00:00:00-04:00,QSM_B15,39108-34-4,8:2FTS,,40,ng/L,U,1900-01-01T00:00:00-05:00,24.0,Yes,pre-treatment
26,123463.0,Air Force,Air Force Plant 44,Active,FA99631,FA99631-1,2022-10-11T00:00:00-04:00,QSM_B15,2991-50-6,NEtFOSAA,,8,ng/L,U,1900-01-01T00:00:00-05:00,24.0,Yes,pre-treatment
27,123464.0,Air Force,Air Force Plant 44,Active,FA99631,FA99631-1,2022-10-11T00:00:00-04:00,QSM_B15,2355-31-9,NMeFOSAA,,8,ng/L,U,1900-01-01T00:00:00-05:00,24.0,Yes,pre-treatment


### Clean and filter data

In [9]:
#determine number of individual bases
bases = final_data_nona.groupby('FinalData.InstallationName', as_index=False).count()
bases.head()

,FinalData.InstallationName,FinalData.Id,FinalData.Component,FinalData.InstallationType,FinalData.LabReportId,FinalData.LaboratorySampleId,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.CasNumber,FinalData.AnalyteAbbrev,FinalData.FinalResult,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.FinalResultDate,FinalData.LkupInstallationId,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,Air Force Plant 44,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
1,Arnold AFB,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
2,Avon Park Air Force Reserve,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
3,BEAUFORT SC MCAS,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
4,BREMERTON WA NAVBASE,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50


In [10]:
#pull out only PFAS chemicals that EPA has released updated standards for
#https://www.epa.gov/sdwa/and-polyfluoroalkyl-substances-pfas
EPA_chemicals = ['PFOA','PFOS','PFNA','PFHxS','PFBS','HFPO-DA'] #HFPO-DA is Gen X
final_data_EPA = final_data_nona[final_data_nona['FinalData.AnalyteAbbrev'].isin(EPA_chemicals)]
final_data_EPA = final_data_EPA.drop(['FinalData.LabReportId',
                                        'FinalData.LaboratorySampleId',
                                        'FinalData.CasNumber',
                                        'FinalData.FinalResultDate',
                                        'FinalData.LkupInstallationId'], 
                                         axis=1)
final_data_EPA.reset_index(drop=True, inplace=True)
final_data_EPA.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123465.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFBS,5.2,4,ng/L,J,Yes,pre-treatment
1,123472.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFHxS,24.3,4,ng/L,,Yes,pre-treatment
2,123475.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFNA,,4,ng/L,U,Yes,pre-treatment
3,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,4,ng/L,,Yes,pre-treatment
4,123478.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,ng/L,J,Yes,pre-treatment


In [11]:
#select only PFOA and PFOS because EPA has proposed standards for these that aren't based on hazard index
PFOS_PFOA = ['PFOA', 'PFOS']
final_PFOS_PFOA = final_data_EPA[final_data_EPA['FinalData.AnalyteAbbrev'].isin(PFOS_PFOA)]
final_PFOS_PFOA.reset_index(drop=True, inplace=True)
final_PFOS_PFOA.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,4,ng/L,,Yes,pre-treatment
1,123478.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,ng/L,J,Yes,pre-treatment
2,123501.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,28.5,4,ng/L,,Yes,pre-treatment
3,123502.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,ng/L,J,Yes,pre-treatment
4,24112.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,13.9,4.2,ng/L,,No,


### Compare samples to EPA proposed standard of 4 ppt

In [12]:
#add column with EPA proposed MCL of 4 ppt (ng/L) (same for both PFOA and PFOS) for comparison
#From EPA: Maximum Contaminant Level (MCL): The highest level of a contaminant that is allowed in drinking water
final_PFOS_PFOA.insert(8, 'MCLs', 4)
final_PFOS_PFOA.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,4,4,ng/L,,Yes,pre-treatment
1,123478.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,4,ng/L,J,Yes,pre-treatment
2,123501.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,28.5,4,4,ng/L,,Yes,pre-treatment
3,123502.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,4,ng/L,J,Yes,pre-treatment
4,24112.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,13.9,4,4.2,ng/L,,No,


In [13]:
#compare sampling results with the MCL
final_PFOS_PFOA = final_PFOS_PFOA.copy()
final_PFOS_PFOA["FinalData.FinalResult"] = pd.to_numeric(final_PFOS_PFOA["FinalData.FinalResult"])
final_PFOS_PFOA.insert(9, "exceedance", final_PFOS_PFOA["FinalData.FinalResult"] > final_PFOS_PFOA["MCLs"])
final_PFOS_PFOA.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,4,True,4,ng/L,,Yes,pre-treatment
1,123478.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,False,4,ng/L,J,Yes,pre-treatment
2,123501.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,28.5,4,True,4,ng/L,,Yes,pre-treatment
3,123502.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,False,4,ng/L,J,Yes,pre-treatment
4,24112.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,13.9,4,True,4.2,ng/L,,No,


In [14]:
#select bases that exceed the standard
exceed = final_PFOS_PFOA.loc[final_PFOS_PFOA['exceedance'] == True]
exceed.reset_index(drop=True, inplace=True)
exceed.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,4,True,4,ng/L,,Yes,pre-treatment
1,123501.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,28.5,4,True,4,ng/L,,Yes,pre-treatment
2,24112.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,13.9,4,True,4.2,ng/L,,No,
3,24113.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOA,39.2,4,True,4.2,ng/L,,No,
4,24136.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,14.3,4,True,4.2,ng/L,,No,


In [15]:
#determine number of bases with exceedances. 42 out of 63 had exceedances
bases_exceed = exceed.groupby('FinalData.InstallationName', as_index=False).count()
bases_exceed.head()

,FinalData.InstallationName,FinalData.Id,FinalData.Component,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,Air Force Plant 44,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,Arnold AFB,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,Avon Park Air Force Reserve,4,4,4,4,4,4,4,4,4,4,4,4,4,4
3,Biddle ANGB (formerly Horsham/Willow Grove ANG),14,14,14,14,14,14,14,14,14,14,14,14,8,8
4,Brunswick NAS,1,1,1,1,1,1,1,1,1,1,1,1,0,0


In [16]:
#determine nubmer of bases with exceedances and no treatment system
exceed_no_treat = final_PFOS_PFOA.loc[final_PFOS_PFOA['FinalData.TreatmentSystem'] == 'No']
exceed_no_treat = exceed_no_treat.groupby('FinalData.InstallationName', as_index=False).count()
exceed_no_treat.head()

,FinalData.InstallationName,FinalData.Id,FinalData.Component,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,Arnold AFB,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,Avon Park Air Force Reserve,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,BEAUFORT SC MCAS,6,6,6,6,6,6,2,6,6,6,6,6,6,6
3,BREMERTON WA NAVBASE,6,6,6,6,6,6,1,6,6,6,6,6,6,6
4,Biddle ANGB (formerly Horsham/Willow Grove ANG),10,10,10,10,10,10,10,10,10,10,10,10,10,10


In [17]:
#store for later use in another notebook
%store exceed

Stored 'exceed' (DataFrame)


In [18]:
final_PFOS_PFOA.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,4,True,4,ng/L,,Yes,pre-treatment
1,123478.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,False,4,ng/L,J,Yes,pre-treatment
2,123501.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,28.5,4,True,4,ng/L,,Yes,pre-treatment
3,123502.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,4,False,4,ng/L,J,Yes,pre-treatment
4,24112.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,13.9,4,True,4.2,ng/L,,No,


### Only keep the highest concentration for each military base for ease of visualization on a map

In [19]:
#sorting by military base name and then PFAS concentration and then only keeping the highest concentration of PFAS for
#each military base
max_final_PFOS_PFOA = final_PFOS_PFOA.sort_values(['FinalData.InstallationName','FinalData.FinalResult'],
                                                  ascending=False).drop_duplicates('FinalData.InstallationName')
max_final_PFOS_PFOA = max_final_PFOS_PFOA.sort_values('FinalData.InstallationName', ascending=True)
max_final_PFOS_PFOA.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
0,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.200,4,True,4,ng/L,,Yes,pre-treatment
7,24137.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOA,40.900,4,True,4.2,ng/L,,No,
8,22261.0,Air Force,Avon Park Air Force Reserve,Active,2021-12-20T00:00:00-05:00,QSM_B15,PFOS,40.300,4,True,4,ng/L,,No,
15,137985.0,Navy,BEAUFORT SC MCAS,Active,2022-12-09T00:00:00-05:00,537.1,PFOA,1.490,4,False,1.19,ng/L,J,No,
39,74213.0,Navy,BREMERTON WA NAVBASE,Active,2022-05-03T00:00:00-04:00,537.1,PFOA,0.478,4,False,1.12,ng/L,J,No,


In [20]:
#optional export sample data table to csv 
#max_final_PFOS_PFOA.to_csv('max_final_PFOS_PFOA.csv', index='False')

### Add location data to the sample data 

In [21]:
#merge sample data and metadata df to attach latitude and longitude data from the metadata to the sample data
max_final_PFOS_PFOA_geo = max_final_PFOS_PFOA.merge(
    metadata_df[['InstallationName', 'State','Latitude','Longitude']]
    .rename({'InstallationName': 'FinalData.InstallationName'},axis=1), 
    on='FinalData.InstallationName', how='left', indicator=True)
#remove extra names from column names
max_final_PFOS_PFOA_geo.columns = max_final_PFOS_PFOA_geo.columns.str.replace('[a-zA-Z]+\\.','', regex=True)
#drop columns that are not needed
max_final_PFOS_PFOA_geo = max_final_PFOS_PFOA_geo.drop(['_merge',
                                                        'Id',
                                                        'InstallationType'], axis=1)

#remove time stamps and format dates
max_final_PFOS_PFOA_geo['SampleDate'] = pd.to_datetime(max_final_PFOS_PFOA_geo.SampleDate, format='%Y-%m-%d %H:%M:%S')
max_final_PFOS_PFOA_geo['SampleDate'] = pd.to_datetime(max_final_PFOS_PFOA_geo['SampleDate'], utc=True)
max_final_PFOS_PFOA_geo['SampleDate'] = max_final_PFOS_PFOA_geo['SampleDate'].dt.strftime('%Y-%m-%d')
# make military base name all uppercase to make name formats consistent
max_final_PFOS_PFOA_geo['InstallationName'] = max_final_PFOS_PFOA_geo['InstallationName'].str.upper()
#change location of the State column
max_final_PFOS_PFOA_geo.insert(0, 'State', max_final_PFOS_PFOA_geo.pop('State'))
#rename column names
max_final_PFOS_PFOA_geo = max_final_PFOS_PFOA_geo.rename(columns={'Component': 'Branch',
                                                                  'InstallationName': 'Name',
                                                                 'SampleDate': 'Date',
                                                                 'AnalysisMethod':'Method',
                                                                 'AnalyteAbbrev':'Analyte',
                                                                 'FinalResult':'Result',
                                                                 'LimitOfDetection':'LimitDetect',
                                                                 'FinalUnits':'Units',
                                                                 'FinalQualification':'Qualifier',
                                                                 'TreatmentSystem':'TreatSystem',
                                                                 'PreOrPostTreatment':'TreatStatus'})

#remove trailing zeros from result column
max_final_PFOS_PFOA_geo["Result"] = max_final_PFOS_PFOA_geo["Result"].apply(
    lambda x:'{:.{}f}'.format(x, (len(str(x)) - 1 - int(str(x).find('.')))) if isinstance(x, float) else x)

max_final_PFOS_PFOA_geo.head()#.dtypes

,State,Branch,Name,Date,Method,Analyte,Result,MCLs,exceedance,LimitDetect,Units,Qualifier,TreatSystem,TreatStatus,Latitude,Longitude
0,Arizona,Air Force,AIR FORCE PLANT 44,2022-10-11,QSM_B15,PFOS,29.2,4,True,4,ng/L,,Yes,pre-treatment,32.221743,-110.926479
1,Tennessee,Air Force,ARNOLD AFB,2021-12-14,QSM_B15,PFOA,40.9,4,True,4.2,ng/L,,No,,35.398810,-86.078066
2,Florida,Air Force,AVON PARK AIR FORCE RESERVE,2021-12-20,QSM_B15,PFOS,40.3,4,True,4,ng/L,,No,,27.595670,-81.506186
3,South Carolina,Navy,BEAUFORT SC MCAS,2022-12-09,537.1,PFOA,1.49,4,False,1.19,ng/L,J,No,,32.475964,-80.726366
4,Washington,Navy,BREMERTON WA NAVBASE,2022-05-03,537.1,PFOA,0.478,4,False,1.12,ng/L,J,No,,47.711998,-122.714478


In [22]:
#optional export sample data table to csv
#max_final_PFOS_PFOA_geo.to_csv('max_final_PFOS_PFOA_geo.csv', index='False')

### Compare sample results to 70 ppt rather than 4 ppt
#### EPA has older guidance of 70 ppt

In [23]:
#applying the 70 ppt standard to PFOS and PFOA data instead of 4 ppt

final_PFOS_PFOA_70 = final_data_EPA.copy()
final_PFOS_PFOA_70 = final_PFOS_PFOA_70[final_PFOS_PFOA_70['FinalData.AnalyteAbbrev'].isin(PFOS_PFOA)]
final_PFOS_PFOA_70.insert(8, 'MCLs', 70)
final_PFOS_PFOA_70["FinalData.FinalResult"] = pd.to_numeric(final_PFOS_PFOA_70["FinalData.FinalResult"])
final_PFOS_PFOA_70.insert(9, "exceedance", final_PFOS_PFOA_70["FinalData.FinalResult"] > final_PFOS_PFOA_70["MCLs"])
#remove trailing zeros
final_PFOS_PFOA_70["FinalData.FinalResult"] = final_PFOS_PFOA_70["FinalData.FinalResult"].apply(
    lambda x:'{:.{}f}'.format(x, (len(str(x)) - 1 - int(str(x).find('.')))) if isinstance(x, float) else x)



final_PFOS_PFOA_70.head()

,FinalData.Id,FinalData.Component,FinalData.InstallationName,FinalData.InstallationType,FinalData.SampleDate,FinalData.AnalysisMethod,FinalData.AnalyteAbbrev,FinalData.FinalResult,MCLs,exceedance,FinalData.LimitOfDetection,FinalData.FinalUnits,FinalData.FinalQualification,FinalData.TreatmentSystem,FinalData.PreOrPostTreatment
3,123477.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,29.2,70,False,4,ng/L,,Yes,pre-treatment
4,123478.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,70,False,4,ng/L,J,Yes,pre-treatment
8,123501.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOS,28.5,70,False,4,ng/L,,Yes,pre-treatment
9,123502.0,Air Force,Air Force Plant 44,Active,2022-10-11T00:00:00-04:00,QSM_B15,PFOA,3.4,70,False,4,ng/L,J,Yes,pre-treatment
13,24112.0,Air Force,Arnold AFB,Active,2021-12-14T00:00:00-05:00,QSM_B15,PFOS,13.9,70,False,4.2,ng/L,,No,


In [24]:
# prepare data for mapping


final_PFOS_PFOA_70 = final_PFOS_PFOA_70.copy()


#deal with nan values in order to create a numeric field
#replace nan values with numpy nan
final_PFOS_PFOA_70['FinalData.FinalResult'].replace(['None', 'nan'], np.nan, inplace=True)
#replace numpy nan with a blank
final_PFOS_PFOA_70['FinalData.FinalResult'] = final_PFOS_PFOA_70['FinalData.FinalResult'].fillna('')
#convert Result to numeric 
final_PFOS_PFOA_70['FinalData.FinalResult'] = pd.to_numeric(final_PFOS_PFOA_70['FinalData.FinalResult'])

final_PFOS_PFOA_70

#sort data by installation name and then by result, then drop duplicates so only keep worst case scenario
max_final_PFOS_PFOA_70 = final_PFOS_PFOA_70.sort_values(['FinalData.InstallationName',
                                                         'FinalData.FinalResult'], 
                                                          ascending=[True, False]).drop_duplicates('FinalData.InstallationName')

max_final_PFOS_PFOA_70 = max_final_PFOS_PFOA_70.sort_values('FinalData.InstallationName', ascending=True)

#merge sample data and metadata df to attach latitude and longitude data from the metadata to the sample data
max_final_PFOS_PFOA_70_geo = max_final_PFOS_PFOA_70.merge(
    metadata_df[['InstallationName', 'State','Latitude','Longitude']]
    .rename({'InstallationName': 'FinalData.InstallationName'},axis=1), 
    on='FinalData.InstallationName', how='left', indicator=True)



#remove "Final Data" from column heaaders
max_final_PFOS_PFOA_70_geo.columns = max_final_PFOS_PFOA_70_geo.columns.str.replace('[a-zA-Z]+\\.','', regex=True)
#drop excess columns
max_final_PFOS_PFOA_70_geo = max_final_PFOS_PFOA_70_geo.drop(['_merge',
                                                       'Id',
                                                       'InstallationType'], axis=1)

#remove timestamp from dates
max_final_PFOS_PFOA_70_geo['SampleDate'] = pd.to_datetime(max_final_PFOS_PFOA_70_geo.SampleDate, format='%Y-%m-%d %H:%M:%S')
max_final_PFOS_PFOA_70_geo['SampleDate'] = pd.to_datetime(max_final_PFOS_PFOA_70_geo['SampleDate'], utc=True)
max_final_PFOS_PFOA_70_geo['SampleDate'] = max_final_PFOS_PFOA_70_geo['SampleDate'].dt.strftime('%Y-%m-%d')
#make names of bases all uppercase
max_final_PFOS_PFOA_70_geo['InstallationName'] = max_final_PFOS_PFOA_70_geo['InstallationName'].str.upper()
#move state column to the front
max_final_PFOS_PFOA_70_geo.insert(0, 'State', max_final_PFOS_PFOA_70_geo.pop('State'))
#rename column names
max_final_PFOS_PFOA_70_geo = max_final_PFOS_PFOA_70_geo.rename(columns={'Component': 'Branch',
                                                                  'InstallationName': 'Name',
                                                                 'SampleDate': 'Date',
                                                                 'AnalysisMethod':'Method',
                                                                 'AnalyteAbbrev':'Analyte',
                                                                 'FinalResult':'Result',
                                                                 'LimitOfDetection':'LimitDetect',
                                                                 'FinalUnits':'Units',
                                                                 'FinalQualification':'Qualifier',
                                                                 'TreatmentSystem':'TreatSystem',
                                                                 'PreOrPostTreatment':'TreatStatus'})

max_final_PFOS_PFOA_70_geo.Result.replace(['None', 'nan'], np.nan, inplace=True)
max_final_PFOS_PFOA_70_geo.Result = max_final_PFOS_PFOA_70_geo.Result.fillna('')
max_final_PFOS_PFOA_70_geo["Result"] = pd.to_numeric(max_final_PFOS_PFOA_70_geo["Result"])

# #remove trailing zeros from result column
max_final_PFOS_PFOA_70_geo["Result"] = max_final_PFOS_PFOA_70_geo["Result"].apply(
     lambda x:'{:.{}f}'.format(x, (len(str(x)) - 1 - int(str(x).find('.')))) if isinstance(x, float) else x)

max_final_PFOS_PFOA_70_geo.head()#.dtypes


,State,Branch,Name,Date,Method,Analyte,Result,MCLs,exceedance,LimitDetect,Units,Qualifier,TreatSystem,TreatStatus,Latitude,Longitude
0,Arizona,Air Force,AIR FORCE PLANT 44,2022-10-11,QSM_B15,PFOS,29.2,70,False,4,ng/L,,Yes,pre-treatment,32.221743,-110.926479
1,Tennessee,Air Force,ARNOLD AFB,2021-12-14,QSM_B15,PFOA,40.9,70,False,4.2,ng/L,,No,,35.398810,-86.078066
2,Florida,Air Force,AVON PARK AIR FORCE RESERVE,2021-12-20,QSM_B15,PFOS,40.3,70,False,4,ng/L,,No,,27.595670,-81.506186
3,South Carolina,Navy,BEAUFORT SC MCAS,2022-12-09,537.1,PFOA,1.49,70,False,1.19,ng/L,J,No,,32.475964,-80.726366
4,Washington,Navy,BREMERTON WA NAVBASE,2022-05-03,537.1,PFOA,0.478,70,False,1.12,ng/L,J,No,,47.711998,-122.714478


In [25]:
#optional export to csv
#max_final_PFOS_PFOA_70_geo.to_csv('max_final_PFOS_PFOA_70_geo.csv', index='False')

#### Look at all the data to determine date range

In [26]:
#analyze datees
final_data_dates = final_data_nona
#remove "Final Data" from column heaaders
final_data_dates.columns = final_data_dates.columns.str.replace('[a-zA-Z]+\.','', regex=True)
#drop excess columns
final_data_dates = final_data_dates.drop(['Id','InstallationType', 'FinalResultDate'], axis=1)
                                          
                                                       
                                                       

#remove timestamp from dates
final_data_dates['SampleDate'] = pd.to_datetime(final_data_dates.SampleDate, format='%Y-%m-%d %H:%M:%S')
final_data_dates['SampleDate'] = pd.to_datetime(final_data_dates['SampleDate'], utc=True)
final_data_dates['SampleDate'] = final_data_dates['SampleDate'].dt.strftime('%Y-%m-%d')
final_data_dates = final_data_dates.sort_values(['SampleDate'], ascending=True)
final_data_dates.reset_index(drop=True, inplace=True)
final_data_dates.head()

,Component,InstallationName,LabReportId,LaboratorySampleId,SampleDate,AnalysisMethod,CasNumber,AnalyteAbbrev,FinalResult,LimitOfDetection,FinalUnits,FinalQualification,LkupInstallationId,TreatmentSystem,PreOrPostTreatment
0,Navy,WHITING FLD FL NAS,2108235,2108235-10,2021-08-23,537.1,2991-50-6,NEtFOSAA,,1.53,ng/L,U,688.0,None,None
1,Navy,WHITING FLD FL NAS,2108235,2108235-09,2021-08-23,537.1,335-67-1,PFOA,,1.51,ng/L,UJ,688.0,None,None
2,Navy,WHITING FLD FL NAS,2108235,2108235-09,2021-08-23,537.1,335-76-2,PFDA,,1.51,ng/L,U,688.0,None,None
3,Navy,WHITING FLD FL NAS,2108235,2108235-09,2021-08-23,537.1,355-46-4,PFHxS,,1.51,ng/L,U,688.0,None,None
4,Navy,WHITING FLD FL NAS,2108235,2108235-09,2021-08-23,537.1,375-73-5,PFBS,,1.51,ng/L,U,688.0,None,None
